IMPORTING THE REQUIRED LIBRARIES

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

MODEL TO FIND FREQUENT ITEM SETS AND RULES.<br/>
MODEL TESTED ON Bakery DATASET.

In [14]:
#FREQUENT ITEMSET METHOD :
#FINDING FREQUENT ITEMSET USING APRIORI ALGORITHMS
def loadDataset():
    #loading the processed_1.csv dataset which is obtained by processing the bakery-transaction dataset
    dataset=pd.read_csv(r'dataset/processed_1.csv')
    transactions=list()
    #converting the transaction to a list and appending it to the transaction list. 
    for i in range(len(dataset.Items)):
        itm=dataset.Items[i].strip().split("|")
        transactions.append(itm)
    #returning the transaction list
    return transactions

#function to get all unique transactions 
def createC1(dataset):
    c1=[]
    for transaction in dataset:
        for item in transaction:
            if not [item] in c1:
                c1.append([item])
    c1.sort()
    return list(map(frozenset,c1))

#function to calculate support of each itemset.
def scand(d,ck,minSupport):
    sscnt={}
    #foreach transaction in the dataset
    for tid in d:
        #for each candidate in the candidate list
        for can in ck:
            #if that candidate is present in the currend transaction 
            if can.issubset(tid):
                #if not there in the dict initialize
                if can not in sscnt.keys():
                    sscnt[can]=1
                #otherwise increment the value of the corresponding key
                else:
                    sscnt[can]+=1
    #finding the total number of trnasactions
    numItems=float(len(d))
    #list to hold the items that satisfies the minimum support
    retList=list()
    #dict to hold the support of various candidates
    supportData={}
    #loop to find the support of each itemset
    for key in sscnt:
        
        support=sscnt[key]/numItems
        if support>=minSupport:
            retList.insert(0,key)
        supportData[key]=support
    #retruning list whcih contains frequent sets. and supports of all the item sets
    return retList,supportData



In [15]:
#loading the dataset
dataset=loadDataset()
c1=createC1(dataset)
l1,supData=scand(dataset,c1,0.1 )

In [16]:
print(l1)
print(supData)


[frozenset({'Cake'}), frozenset({'Tea'}), frozenset({'Coffee'}), frozenset({'Bread'})]
{frozenset({'Bread'}): 0.32720549392498677, frozenset({'Scandinavian'}): 0.029054410987849975, frozenset({'Cookies'}): 0.054410987849973586, frozenset({'Hot chocolate'}): 0.05832012678288431, frozenset({'Jam'}): 0.015002641310089805, frozenset({'Muffin'}): 0.03845747490755415, frozenset({'Coffee'}): 0.47839408346539886, frozenset({'Pastry'}): 0.0861067089276281, frozenset({'Medialuna'}): 0.061806656101426306, frozenset({'Tea'}): 0.14263074484944532, frozenset({'Tartine'}): 0.004860010565240359, frozenset({'Basket'}): 0.0006339144215530904, frozenset({'Mineral water'}): 0.014157422081352351, frozenset({'Farm House'}): 0.03919704173269942, frozenset({'Fudge'}): 0.015002641310089805, frozenset({'Juice'}): 0.03856312731114633, frozenset({"Ella's Kitchen Pouches"}): 0.0017960908610670893, frozenset({'Victorian Sponge'}): 0.0007395668251452721, frozenset({'Frittata'}): 0.00855784469096672, frozenset({'Hear

In [17]:
#function ro find support for given length of itemsets
def aprioriGen(lk,k):
    retList=[]
    lenlk=len(lk)
    for i in range(lenlk):
        for j in range(i+1,lenlk):
            #join sets if first k-2 items are equal
            l1=list(lk[i])[:k-2]
            l2=list(lk[j])[:k-2]
            l1.sort()
            l2.sort()
            if l1==l2:
                retList.append(lk[i] | lk[j])
    return retList
def apriori(dataset,minsupport=0.2):
    #generating candidate itmes
    c1=createC1(dataset)
    #frequent itemssets with one item
    l1,supportData=scand(dataset,c1,minsupport)
    
    l=[l1]
    #starting to generate fequent itemset from two items onwards
    k=2
    while(len(l[k-2])>0):
        #generating candidate sets with k items
        ck=aprioriGen(l[k-2],k)
        
        lk,supk=scand(dataset,ck,minsupport)
        supportData.update(supk)
        l.append(lk)
        k+=1
    return l,supportData


In [18]:
l,supportData=apriori(dataset,minsupport=0.07)
for i in range(len(l)):
    print(l[i])
    print()

[frozenset({'Sandwich'}), frozenset({'Cake'}), frozenset({'Tea'}), frozenset({'Pastry'}), frozenset({'Coffee'}), frozenset({'Bread'})]

[frozenset({'Bread', 'Coffee'})]

[]



In [19]:
#function to generate apriori rules.
def generateRules(L,supportData,minconf=0.5):
    bigRuleList=list()
    for i in range(1,len(L)):
        for freqSet in L[i]:
            H1=[frozenset([item] for item in freqSet)]
            if(i>1):
                rulesFromConseq(freqSet,H1,supportData,bigRuleList,minconf)
            else:
                calcConf(freqSet,H1,supportData,bigRuleList,minconf)
    return bigRuleList
def calcConf(freqSet,H,supportData,brl,minconf=0.5):
    prunedH=list()
    for conseq in H:
        conf=supportData[freqSet]/supportData[freqSet-conseq]
        if cong>=minconf:
            print(freqSet-conseq,'--->',conseq,'conf:',conf)
            brl.append((freqSet-conseq,conseq,conf))
            prunedH.append(conseq)
    return prunedH
def rulesFromConseq(freqSet,H,supportData,brl,minconf=0.5):
    m=len(H[0])
    if(len(freqSet)>(m+1)):
        Hmp1=aprioriGen(H,m+1)
        Hmp1=calcConf(freqSet,Hmp1,supportData,br1,minconf)
        if(len(Hmp1)>1):
            rulesFromConseq(freqSet,Hmp1,supportData,brl,minconf)
    

In [20]:
dataset=loadDataset()
c1=createC1(dataset)
D=map(set,dataset)

L,suppData=apriori(dataset)
rules=generateRules(L,suppData,minconf=0.00007)
print(rules)
#the dataset being so sparse that it fails to generate any apriori rules eventhough the minconf is so small.

[]
